In [1]:
import datetime
import time
import pytz

import pandas as pd
import numpy as np
import pandas_market_calendars as pmc

import sys, os
sys.path.append(os.path.join(os.path.dirname('.'), '..', 'lib'))
sys.path.insert(0, '/Users/chris/programming/interactivebrokers')
import ibapi

sys.path.insert(0, '/Users/chris/programming/analyses/trading')
import tradingutils as tutils

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [2]:
import master
import marketdata
import contracts
import helper

PORT = 7497
md_app = marketdata.get_instance(port=PORT)
ct_app = contracts.get_instance(port=PORT)

ERROR -1 2104 Market data farm connection is OK:usfuture.nj
ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:euhmds
ERROR -1 2106 HMDS data farm connection is OK:fundfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefnj


serverVersion:155 connectionTime:b'20200423 01:00:11 EST'
MarketDataApp connecting to IB...
ERROR: request -1
ERROR: request -1
ERROR: request -1
ERROR: request -1
ERROR: request -1
ERROR: request -1
ERROR: request -1
ERROR: request -1
ERROR: request -1


ERROR -1 2104 Market data farm connection is OK:usfuture.nj
ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:euhmds
ERROR -1 2106 HMDS data farm connection is OK:fundfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefnj


MarketDataApp connected.
serverVersion:155 connectionTime:b'20200423 01:00:11 EST'
MarketDataApp connecting to IB...
ERROR: request -1
ERROR: request -1
ERROR: request -1
ERROR: request -1
ERROR: request -1
ERROR: request -1
ERROR: request -1
ERROR: request -1
ERROR: request -1
MarketDataApp connected.


In [67]:
# Find a particular contract
partial_contract = ibapi.contract.Contract()
partial_contract.symbol = 'CL'
partial_contract.secType = 'FUT'
partial_contract.lastTradeDateOrContractMonth = '202006'
partial_contract.currency = 'USD'

#ct_app.match_contract(partial_contract)
matching_contracts = ct_app.get_all_matching_contracts(partial_contract)
[print('Match {}: {}'.format(j, mc)) for j, mc in enumerate(matching_contracts)]

_contract = matching_contracts[0]

#ct_app.add_to_saved_contracts([_contract])

print('Saved: {}'.format(ct_app.get_contract(_contract.localSymbol)))

Match 0: 97589608,CL,FUT,20200519,0.0,,1000,NYMEX,,USD,CLM0,CL,False,,combo:
Match 1: 97589608,CL,FUT,20200519,0.0,,1000,QBALGO,,USD,CLM0,CL,False,,combo:
Match 2: 388005644,CL,FUT,20200619,0.0,,100,ONE,,USD,CL1DM0,CL1D,False,,combo:
Saved: 97589608,CL,FUT,20200519,0.0,,1000,NYMEX,,USD,CLM0,CL,False,,combo:


In [18]:
# First, create a historical time series request
ticker = 'ESM0'
frequency = '5s'
data_type = 'TRADES'
start = ''
end = ''
duration = '60s'

contractList = [ct_app.get_contract(ticker)]
histReqObjList = md_app.create_historical_data_request(contractList, is_snapshot=False, frequency=frequency, 
                                        use_rth=False, data_type=data_type,
                                        start=start, end=end, duration=duration)

In [19]:
# Place the time series request
histReqObj = histReqObjList[0]
histReqObj.place_request()

In [20]:
hist_df = histReqObj.get_dataframe()
hist_df.tail()

,close,date,high,low,open
est_timestamp,,,,,
1.587597e+09,2785.25,20200423 01:01:50,2785.25,2785.25,2785.25
1.587597e+09,2785.75,20200423 01:02:00,2786.00,2785.25,2785.25
1.587597e+09,2785.75,20200423 01:02:05,2785.75,2785.75,2785.75
1.587597e+09,2785.50,20200423 01:02:10,2785.75,2785.50,2785.75
1.587597e+09,2785.25,20200423 01:02:15,2785.50,2785.25,2785.50


In [3]:
# Get streaming 5 second bars
ticker = 'ESM0'
data_type = 'TRADES'

contractList = [ct_app.get_contract(ticker)]
streamingBarObj = md_app.create_streaming_bar_data_request(contractList, use_rth=False, data_type=data_type)

In [4]:
strBarObj = streamingBarObj[0]
strBarObj.place_request()

In [33]:
# Create a time series request for ticks
ticker = 'ESM0'
data_type = 'Last'
number_of_ticks = 0

contractList = [ct_app.get_contract(ticker)]
tickReqObjList = md_app.create_streaming_tick_data_request(contractList, data_type=data_type,
                                                    number_of_ticks=number_of_ticks)

In [34]:
tickReqObj = tickReqObjList[0]
tickReqObj.place_request()

In [41]:
df_tick = tickReqObj.get_dataframe()
df_tick.tail()

,price,size
time,,
1587618580,2792.75,1
1587618583,2792.50,4
1587618583,2792.50,1
1587618583,2792.50,1
1587618583,2792.75,5


ERROR -1 1100 Connectivity between IB and Trader Workstation has been lost.
ERROR -1 1100 Connectivity between IB and Trader Workstation has been lost.


ERROR: request -1
ERROR: request -1


In [40]:
ts = tutils.convert_tick_to_time_bars(df_tick)
ts.tail()

,open,high,low,close,volume,barCount,average
1587618265,2791.75,2792.00,2791.75,2791.75,8,4,2791.906250
1587618266,2792.00,2792.00,2791.75,2792.00,3,3,2791.916667
1587618267,2792.00,2792.00,2792.00,2792.00,15,8,2792.000000
1587618268,2791.75,2791.75,2791.75,2791.75,24,10,2791.750000
1587618269,2791.50,2791.50,2791.25,2791.50,21,8,2791.476190


In [637]:
#df.to_hdf('data/ESM0_tick.h5', 'TRADES_20200422')
#new_df = pd.read_hdf('data/ESM0_tick.h5', 'TRADES_20200422')

ERROR -1 1100 Connectivity between IB and Trader Workstation has been lost.
ERROR -1 1100 Connectivity between IB and Trader Workstation has been lost.


ERROR: request -1
ERROR: request -1
